In [1]:
from geopy.geocoders import ArcGIS
import pandas as pd
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

Working with GeoPy and Folium

In [2]:
addr="Champ de Mars, Paris, France"

nom=ArcGIS()
n=nom.geocode(addr)

print('Latitude = {}, Longitude = {}'.format(n.latitude, n.longitude))

Latitude = 48.856130000000064, Longitude = 2.2980300000000398


In [3]:
df=pd.read_csv('addresses.csv')
df.drop(['Unnamed: 0'],axis=1,inplace=True)
df.head()

Typ   Nr          Namn           Address1 Address3   Address4  \
0  Butik  102  Fältöversten       Karlaplan 13   115 20  STOCKHOLM   
1  Butik  104           NaN      Nybrogatan 47   114 39  STOCKHOLM   
2  Butik  106    Garnisonen   Karlavägen 100 A   115 26  STOCKHOLM   
3  Butik  110           NaN      Hötorgshallen   111 57  STOCKHOLM   
4  Butik  113        Sergel  Drottninggatan 45   111 21  STOCKHOLM   

         Address5       Telefon  
0  Stockholms län  08/662 22 89  
1  Stockholms län  08/662 50 16  
2  Stockholms län  08/662 64 85  
3  Stockholms län   08/56849241  
4  Stockholms län   08/21 47 44

In [4]:
df['ADDRESS']=df.apply(lambda x:x['Address1']+','+x['Address3']+','+x['Address4']+','+x['Address5'],axis=1)
df.head()

Typ   Nr          Namn           Address1 Address3   Address4  \
0  Butik  102  Fältöversten       Karlaplan 13   115 20  STOCKHOLM   
1  Butik  104           NaN      Nybrogatan 47   114 39  STOCKHOLM   
2  Butik  106    Garnisonen   Karlavägen 100 A   115 26  STOCKHOLM   
3  Butik  110           NaN      Hötorgshallen   111 57  STOCKHOLM   
4  Butik  113        Sergel  Drottninggatan 45   111 21  STOCKHOLM   

         Address5       Telefon  \
0  Stockholms län  08/662 22 89   
1  Stockholms län  08/662 50 16   
2  Stockholms län  08/662 64 85   
3  Stockholms län   08/56849241   
4  Stockholms län   08/21 47 44   

                                             ADDRESS  
0       Karlaplan 13,115 20,STOCKHOLM,Stockholms län  
1      Nybrogatan 47,114 39,STOCKHOLM,Stockholms län  
2   Karlavägen 100 A,115 26,STOCKHOLM,Stockholms län  
3      Hötorgshallen,111 57,STOCKHOLM,Stockholms län  
4  Drottninggatan 45,111 21,STOCKHOLM,Stockholms län

In [5]:
from geopy.extra.rate_limiter import RateLimiter

# conveneint function to delay between geocoding calls
geocode = RateLimiter(nom.geocode, min_delay_seconds=1)

# create location column
df['location'] = df['ADDRESS'].apply(geocode)

# create longitude, laatitude and altitude from location column (returns tuple)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

# split point column into latitude, longitude and altitude columns
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)

In [7]:
df = df.drop(['Address1', 'Address3', 'Address4', 'Address5', 'Telefon', 'ADDRESS', 'location', 'point'], axis=1)
df.head()

Typ   Nr          Namn  latitude  longitude  altitude
0  Butik  102  Fältöversten  59.33880   18.09087       0.0
1  Butik  104           NaN  59.33713   18.07907       0.0
2  Butik  106    Garnisonen  59.33604   18.09650       0.0
3  Butik  110           NaN  59.33438   18.06239       0.0
4  Butik  113        Sergel  59.33245   18.06224       0.0

In [8]:
locationlist = df[["latitude","longitude"]].values.tolist()
locationlist

[[59.338799994130994, 18.090869997670353],
 [59.33712998374233, 18.079070038198978],
 [59.33604000105339, 18.09650003821426],
 [59.33438000000007, 18.06239000000005],
 [59.33244999001383, 18.062240014813028],
 [59.33321001908425, 18.07116003617233],
 [59.33041000241897, 18.06500000789063],
 [59.32806998459998, 18.017809976831927],
 [59.32827001871925, 18.042350010034795],
 [59.337239996221484, 18.00939999428084],
 [59.334890003937716, 18.030149982324872],
 [59.34076999283431, 18.041799989548423],
 [59.3463999914687, 18.033290010892443],
 [59.34350999507285, 18.052859994607445],
 [59.344750013828616, 18.06349000803357],
 [59.33914000603329, 18.060460033854696],
 [59.33443997955621, 18.05564999489843],
 [59.30830000867522, 18.075949958560955],
 [59.31426000283764, 18.07514001525712],
 [59.31465998725716, 18.071130028958947],
 [59.31565001575075, 18.084479969965514],
 [59.30985596001992, 18.022039182688534],
 [59.316709991226304, 18.05598996489121],
 [59.30231000729944, 18.103430028121664

### Folium Marker map

Quick Start: https://python-visualization.github.io/folium/quickstart.html

In [9]:
import folium
from folium.plugins import FastMarkerCluster

mapex = folium.Map(
    location=[59.338315,18.089960],
    tiles='cartodbpositron',
    zoom_start=12,
)

locationlist = df[["latitude","longitude"]].values.tolist()
for point in locationlist:
    folium.CircleMarker(point).add_to(mapex)

mapex

Add marker clusters to a map using in-browser rendering. Using FastMarkerCluster it is possible to render 000’s of points far quicker than the MarkerCluster class.

In [10]:
from folium.plugins import MarkerCluster

mapex = folium.Map(
    location=[59.338315,18.089960],
    tiles='cartodbpositron',
    zoom_start=12,
)

marker_cluster = MarkerCluster().add_to(mapex)

locationlist = df[["latitude","longitude"]].values.tolist()

for i,point in enumerate(locationlist):
    folium.Marker(
        location=point,
        popup=df.iloc[i]['Namn'],
        icon=folium.Icon(color='green', icon='ok-sign'),
    ).add_to(marker_cluster)
    
    
mapex

`FastMarkerCluster` is not as flexible as MarkerCluster but, like the name suggests, it is faster.

In [11]:
mapex = folium.Map(
    location=[59.338315,18.089960],
    tiles='cartodbpositron',
    zoom_start=12,
)


FastMarkerCluster(data=list(zip(df['latitude'].values, df['longitude'].values))).add_to(mapex)
mapex

#### Add tile layers

In [12]:
mapex = folium.Map(
    location=[59.338315,18.089960],
    tiles='cartodbpositron',
    zoom_start=12,
)

marker_cluster = MarkerCluster(name='mic').add_to(mapex)

locationlist = df[["latitude","longitude"]].values.tolist()


for i,point in enumerate(locationlist):
    folium.Marker(
        location=point,
        tooltip = "toolip number".format(i),
        popup=df.iloc[i]['Namn'],
        icon=folium.Icon(color='green', icon='ok-sign'),
    ).add_to(marker_cluster)
    

folium.TileLayer('openstreetmap').add_to(mapex)
folium.TileLayer('Stamen Terrain').add_to(mapex) 
folium.LayerControl().add_to(mapex)    
mapex

Choropleth can be easily created by binding the data between Pandas DataFrames/Series and <a href="https://github.com/python-visualization/folium/blob/master/examples/data/us-states.json">Geo/TopoJSON</a> geometries. Color Brewer sequential color schemes are built-in to the library, and can be passed to quickly visualize different combinations. 

In [13]:
import pandas as pd


url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
state_geo = f'{url}/us-states.json'
state_unemployment = f'{url}/US_Unemployment_Oct2012.csv'
state_data = pd.read_csv(state_unemployment)

state_data.head()
#state_geo

State  Unemployment
0    AL           7.1
1    AK           6.8
2    AZ           8.1
3    AR           7.2
4    CA          10.1

In [14]:
m = folium.Map(location=[48, -102], zoom_start=3)

folium.Choropleth(
    geo_data=state_geo,
    name='choropleth',
    data=state_data,
    columns=['State', 'Unemployment'],
    key_on='feature.id',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Unemployment Rate (%)'
).add_to(m)

folium.LayerControl().add_to(m)

m

In [15]:
state_data['poverty'] = 20-state_data['Unemployment']
folium.Map(location=[48, -102], zoom_start=3)\
.add_child(folium.Choropleth(
    geo_data=state_geo,
    name='Unemployment',
    data=state_data,
    columns=['State', 'Unemployment'],
    key_on='feature.id',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Unemployment Rate (%)',reset=True))\
.add_child(folium.Choropleth(
    geo_data=state_geo,
    name='Poverty',
    data=state_data,
    columns=['State', 'poverty'],
    key_on='feature.id',
    fill_color='Reds',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Poverty Rate (%)',reset=True))\
.add_child(folium.LayerControl())

Using FeatureGroup to merge diffrent maps without mutiple legends

In [16]:
df_addresses=pd.read_csv("df_addresses.csv")
df_addresses.set_index("index",inplace=True)

mapa = folium.Map(location=[35.11567262307692,-89.97423444615382], zoom_start=12,
           tiles='Stamen Terrain')

for grp_name, df_grp in df_addresses.groupby('Group'):
    feature_group = folium.FeatureGroup(grp_name)
    for row in df_grp.itertuples():
        folium.Marker(location=[row.Latitude, row.Longitude]).add_to(feature_group)
    feature_group.add_to(mapa)

folium.LayerControl().add_to(mapa)
mapa

In [17]:
from branca.colormap import linear

m=folium.Map(location=[48, -102], zoom_start=3, tiles=None,overlay=False)

unp = state_data[['State','Unemployment']]
pov = state_data[['State','poverty']]

feature_group0 = folium.FeatureGroup(name='unemployment',overlay=False).add_to(m)
feature_group1= folium.FeatureGroup(name='poverty',overlay=False).add_to(m)

fs = [feature_group0,feature_group1]
commodities = [unp,pov]
names=['Unemployment','poverty']

for i in range(len(commodities)): 
    choropleth1 = folium.Choropleth(
    geo_data=state_geo,
    name='choropleth',
    data=commodities[i],
    key_on='feature.id',
    fill_color='YlGn',
    nan_fill_color="black",
    columns=['State', names[i]],
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Arrival (in Quintal)',
    highlight=True,
    line_color='black').geojson.add_to(fs[i])
    
    
    
    geojson1 = folium.GeoJson(data=state_geo,
           name='States',smooth_factor=2,
           style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
           tooltip=folium.GeoJsonTooltip(fields=['name'],labels=False,sticky=True),
           highlight_function=lambda x: {'weight':3,'fillColor':'grey'}).add_to(choropleth1)

    
colormap = linear.YlGn_09.scale(0, 20).to_step(10)
colormap.caption = 'Percentage'
colormap.add_to(m)

folium.TileLayer('cartodbdark_matter',overlay=True,name="dark mode").add_to(m)
folium.TileLayer('cartodbpositron',overlay=True,name="light mode").add_to(m)

folium.LayerControl(collapsed=False).add_to(m)

m

`GeoJson` and `TopoJson` features accepts style_function to allow for further custimization of the map. 

In [18]:
import branca
import requests
import json


url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
county_data = f'{url}/us_county_data.csv'
county_geo = f'{url}/us_counties_20m_topo.json'

json.loads(requests.get(county_geo).text)

{'type': 'Topology',
 'transform': {'scale': [0.035896170617061705, 0.005347309530953095],
  'translate': [-179.14734, 17.884813]},
 'objects': {'us_counties_20m': {'type': 'GeometryCollection',
   'geometries': [{'type': 'Polygon',
     'arcs': [[0, 1, 2, 3, 4]],
     'id': '0500000US01001'},
    {'type': 'Polygon', 'arcs': [[5, 6, 7, 8, 9, 10]], 'id': '0500000US01009'},
    {'type': 'Polygon',
     'arcs': [[11, 12, 13, 14, 15]],
     'id': '0500000US01017'},
    {'type': 'Polygon',
     'arcs': [[16, 17, -3, 18, 19, 20, 21]],
     'id': '0500000US01021'},
    {'type': 'Polygon', 'arcs': [[22, 23, 24, 25]], 'id': '0500000US01033'},
    {'type': 'Polygon',
     'arcs': [[26, 27, 28, 29, 30, 31]],
     'id': '0500000US01045'},
    {'type': 'Polygon',
     'arcs': [[32, 33, 34, -4, -18, 35]],
     'id': '0500000US01051'},
    {'type': 'Polygon',
     'arcs': [[36, 37, 38, 39, 40]],
     'id': '0500000US01065'},
    {'type': 'Polygon',
     'arcs': [[41, 42, 43, 44, -25, 45, 46]],
     '

In [19]:
df = pd.read_csv(county_data, na_values=[' '])

colorscale = branca.colormap.linear.YlOrRd_09.scale(0, 50e3)
employed_series = df.set_index('FIPS_Code')['Employed_2011']


def style_function(feature):
    employed = employed_series.get(int(feature['id'][-5:]), None)
    return {
        'fillOpacity': 0.5,
        'weight': 0,
        'fillColor': '#black' if employed is None else colorscale(employed)
    }


m = folium.Map(
    location=[48, -102],
    tiles='cartodbpositron',
    zoom_start=4
)

folium.TopoJson(
    json.loads(requests.get(county_geo).text),
    'objects.us_counties_20m',
    style_function=style_function
).add_to(m)


m

#### More Examples: 
##### https://medium.com/@saidakbarp/interactive-map-visualization-with-folium-in-python-2e95544d8d9b
##### https://github.com/rsandstroem/IPythonNotebooks/blob/master/GeoMapsFoliumDemo/GeoMapsFoliumDemo.ipynb
##### https://nbviewer.jupyter.org/gist/talbertc-usgs/18f8901fc98f109f2b71156cf3ac81cd